In [1]:
import argparse
import datetime
import os
import sys
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.backends.cudnn as cudnn

from timm.models import create_model

from engine import train_one_epoch, evaluate
from utils import get_training_dataloader, get_test_dataloader
import models

In [5]:
CIFAR100_TRAIN_MEAN = (0.5070751592371323, 0.48654887331495095, 0.4409178433670343)
CIFAR100_TRAIN_STD = (0.2673342858792401, 0.2564384629170883, 0.27615047132568404)
CHECKPOINT_PATH = './checkpoint'
MODEL_NAME = 'vit_tiny_patch16_224'
# MODEL_NAME = 'vit_small_patch16_224'
# MODEL_NAME = 'vit_base_patch16_224'
EPOCHS = 50
LR = 0.0001
WD = 0.0001


print(f"Creating model: {MODEL_NAME}")
model = create_model(
        MODEL_NAME,
        pretrained=True,
        num_classes=100,
        img_size=224)
device = 'cuda:0' # device = 'cpu'
model = model.to(device)

cifar100_training_loader = get_training_dataloader(
    CIFAR100_TRAIN_MEAN,
    CIFAR100_TRAIN_STD,
    num_workers=4,
    batch_size=256,
    shuffle=True
)

cifar100_test_loader = get_test_dataloader(
    CIFAR100_TRAIN_MEAN,
    CIFAR100_TRAIN_STD,
    num_workers=4,
    batch_size=256,
    shuffle=False
)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WD)

n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('number of params:', n_parameters)


Creating model: vit_base_patch16_224


Downloading: "https://dl.fbaipublicfiles.com/deit/deit_base_patch16_224-b5f2ef4d.pth" to /home/soroush/.cache/torch/hub/checkpoints/deit_base_patch16_224-b5f2ef4d.pth


  0%|          | 0.00/330M [00:00<?, ?B/s]

Files already downloaded and verified
Files already downloaded and verified
number of params: 85875556


In [ ]:
print(f"Start training for {EPOCHS} epochs")

for epoch in range(1, EPOCHS+1):
    train_stats = train_one_epoch(
        model, criterion, cifar100_training_loader,
        optimizer, device, epoch)
    test_stats = evaluate(cifar100_test_loader, model, criterion, device)
    print(f"Accuracy of the network on the {len(cifar100_test_loader)} test images: {test_stats['acc1']:.1f}%")        

Start training for 50 epochs
Epoch: [1]  [  0/196]  eta: 0:03:59  loss: 4.7447 (4.7447)  time: 1.2195  data: 0.8094  max mem: 25693
Epoch: [1]  [100/196]  eta: 0:00:42  loss: 2.8646 (3.7247)  time: 0.4420  data: 0.0292  max mem: 26685
Epoch: [1]  [195/196]  eta: 0:00:00  loss: 1.7908 (2.9429)  time: 0.4333  data: 0.0296  max mem: 26685
Epoch: [1] Total time: 0:01:27 (0.4454 s / it)
Averaged stats: loss: 1.7908 (2.9429)
Test:  [ 0/40]  eta: 0:00:34  loss: 1.5626 (1.5626)  acc1: 57.8125 (57.8125)  acc5: 88.2812 (88.2812)  time: 0.8700  data: 0.7121  max mem: 26685
Test:  [20/40]  eta: 0:00:04  loss: 1.6668 (1.6316)  acc1: 55.8594 (56.5290)  acc5: 85.5469 (85.8631)  time: 0.1870  data: 0.0295  max mem: 26685
Test:  [39/40]  eta: 0:00:00  loss: 1.6303 (1.6271)  acc1: 56.2500 (56.5900)  acc5: 85.5469 (86.1000)  time: 0.1798  data: 0.0291  max mem: 26685
Test: Total time: 0:00:08 (0.2031 s / it)
* Acc@1 56.590 Acc@5 86.100 loss 1.627
Accuracy of the network on the 40 test images: 56.6%
Epoch

In [ ]:
start_time = time.time()
test_stats = evaluate(cifar100_test_loader, model, criterion, device)
end_time = time.time()
num_samples = len(cifar100_test_loader.dataset)
throughput = num_samples / (end_time - start_time)
print("Throughput: {}".format(throughput))